# Scraping the Election Results from the CQ Press Library Website

All the congressional election results data we need is stored on the CQ Press Library Website. The website is behind a username/password wall (our Georgetown credentials), so for this notebook we will use one of the html pages that was previously saved. 

This notebook will walk through the process of scraping the html page, storing the relevant information as a dataframe, initial required cleaning, wrangling, and munging, and saving as a .csv so that it is ready to be ingested into the database.

## Getting the data.

The data we are looking for is stored by congressional election year, by state, on the CQ Press Library Website. We need data for the 2012 and 2014 election years, in each of the fifty states. Each webpage includes the election information for a single state, for both the 2012 and 2014 cycles. 

In [1]:
# import the necessary libraries
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# create handle for BeautifulSoup instance
soup = BeautifulSoup(open("./Data/CQ Voting and Elections Collection.html"), "html.parser")

In [3]:
print(soup.get_text())




CQ Voting and Elections Collection






<!--
function popupDocLink(querystring)
{
  // we no longer use a popup window, so for any old links this opens them in the current window
  sid = '';
  if (sid.length > 0)
    sid = '&' + sid;

  document.location.href = 'document.php?' + querystring + sid;
}

function xsite_popupDoc(path, querystring)
{
  document.location.href = path + (path.length ? '/' : '') + 'document.php?' + querystring;
  return false;
}

function popupLegal(path)
{
  legalWindow = window.open(path, 'cqel_legalWin', 'scrollbars=yes,toolbar=no,menubar=no,status=no,resizable=yes,width=600,height=300,top=40,left=40,screenY=40,screenX=40');
  if (legalWindow)
    legalWindow.focus();

  return false;
}

function doOnLoad()
{
  setTimeout("window.location.href='/xsite/logout.php?source=elections&action=auto&'", 3300000);
  }
//-->


	  var _gaq = _gaq || [];
	  _gaq.push(['_setAccount', 'UA-24103675-1']);
	  _gaq.push(['_trackPageview']);

	  (function() {
		var ga = docu

So now we have our soup. Next we need to pull all the table headers and table data from the page. We can find out where the data that we need is by inspecting the webpage itself, which shows that what we want is under the the <td> headers. 

In [5]:
table = soup.findAll('td')

In [6]:
#take a look at what we've extracted

table

[<td valign="top"> </td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Republican</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Democratic</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Highest Other</span></td>,
 <td align="center" valign="top"><span class="tablehead">Other</span></td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td align="center" colspan="4" valign="top"><span class="tablehead">Total Vote (%)</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Major Party Vote (%)</span></td>,
 <td align="center" valign="top"><span class="tablehead">Year</span></td>,
 <td align="center" valign="top"><span class="tablehead">CD</span></td>,
 <td align="center" valign="top"><span class="tablehead">Redistricted Date</span></td>,
 <td align="center" valign="top"><span class="tabl

In [7]:
#pull out just the first row to see the headers

soup.findAll('tr')[1].findAll('td')

[<td valign="top"> </td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Republican</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Democratic</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Highest Other</span></td>,
 <td align="center" valign="top"><span class="tablehead">Other</span></td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td align="center" colspan="4" valign="top"><span class="tablehead">Total Vote (%)</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Major Party Vote (%)</span></td>]

In [8]:
#but the second row of headers are what we actually care about. 

soup.findAll('tr')[2].findAll('td')

[<td align="center" valign="top"><span class="tablehead">Year</span></td>,
 <td align="center" valign="top"><span class="tablehead">CD</span></td>,
 <td align="center" valign="top"><span class="tablehead">Redistricted Date</span></td>,
 <td align="center" valign="top"><span class="tablehead">Total Vote</span></td>,
 <td align="center" valign="top"><span class="tablehead">Vote</span></td>,
 <td align="center" valign="top"><span class="tablehead">Candidate</span></td>,
 <td align="center" valign="top"><span class="tablehead">Vote</span></td>,
 <td align="center" valign="top"><span class="tablehead">Candidate</span></td>,
 <td align="center" valign="top"><span class="tablehead">Vote</span></td>,
 <td align="center" valign="top"><span class="tablehead">Candidate</span></td>,
 <td align="center" valign="top"><span class="tablehead">Other Vote</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Plurality</span></td>,
 <td align="center" valign="top"><span class=

In [9]:
column_headers = [td.getText() for td in 
                  soup.findAll('tr')[2].findAll('td')]

In [10]:
column_headers

['Year',
 'CD',
 'Redistricted Date',
 'Total Vote',
 'Vote',
 'Candidate',
 'Vote',
 'Candidate',
 'Vote',
 'Candidate',
 'Other Vote',
 'Plurality',
 'Rep.',
 'Dem.',
 'Highest Other',
 'Other',
 'Rep.',
 'Dem.']

In [11]:
data_rows = soup.findAll('tr')[3:]

In [12]:
data_rows

[<tr> <td align="left" valign="top"><span class="tabletext">2014</span></td> <td align="right" valign="top"><span class="tabletext">District 1</span></td> <td align="right" valign="top"><span class="tabletext">01/01/2012</span></td> <td align="right" valign="top"><span class="tabletext">152,234</span></td> <td align="right" valign="top"><span class="tabletext">103,758</span></td> <td align="right" valign="top"><span class="tabletext">Byrne, Bradley [<a href="http://library.cqpress.com.proxy.library.georgetown.edu/elections/document.php?id=avg2012-3AL3&amp;type=toc&amp;num=9676B457-9FD2-4541-985C-45CAE50EE210#incumfn">*</a>]</span></td> <td align="right" valign="top"><span class="tabletext">48,278</span></td> <td align="right" valign="top"><span class="tabletext">LeFlore, Burton R.</span></td> <td align="right" valign="top"><span class="tabletext">198</span></td> <td align="right" valign="top"><span class="tabletext">Write-In(Write-in)</span></td> <td align="right" valign="top"><span cl

In [13]:
type(data_rows)

list

In [14]:
race_data = [[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

In [15]:
race_data

[['2014',
  'District 1',
  '01/01/2012',
  '152,234',
  '103,758',
  'Byrne, Bradley [*]',
  '48,278',
  'LeFlore, Burton R.',
  '198',
  'Write-In(Write-in)',
  'N/A',
  '55,480',
  'R',
  '68.16',
  '31.71',
  '0.13',
  '0',
  '68.25',
  '31.75'],
 ['2012',
  'District 1',
  '01/01/2012',
  'Unopposed',
  '196,374',
  'Bonner, Josiah Robbins Jr. [*]',
  'N/A',
  'N/A',
  '4,302',
  'Scattered Write-In(Write-in)',
  'N/A',
  '192,072',
  'R',
  '97.86',
  '0',
  '2.14',
  '0',
  '100',
  '0'],
 ['2014',
  'District 2',
  '01/01/2012',
  '167,952',
  '113,103',
  'Roby, Martha [*]',
  '54,692',
  'Wright, Erick',
  '157',
  'Write-In(Write-in)',
  'N/A',
  '58,411',
  'R',
  '67.34',
  '32.56',
  '0.09',
  '0',
  '67.41',
  '32.59'],
 ['2012',
  'District 2',
  '01/01/2012',
  '283,953',
  '180,591',
  'Roby, Martha [*]',
  '103,092',
  'Ford, Therese',
  '270',
  'Scattered Write-In(Write-in)',
  'N/A',
  '77,499',
  'R',
  '63.6',
  '36.31',
  '0.1',
  '0',
  '63.66',
  '36.34'],
 [

In [ ]:
df = pd.DataFrame(race_data, columns=column_headers)

In [16]:
column_headers.insert(12, "WinningParty")

In [17]:
column_headers

['Year',
 'CD',
 'Redistricted Date',
 'Total Vote',
 'Vote',
 'Candidate',
 'Vote',
 'Candidate',
 'Vote',
 'Candidate',
 'Other Vote',
 'Plurality',
 'WinningParty',
 'Rep.',
 'Dem.',
 'Highest Other',
 'Other',
 'Rep.',
 'Dem.']

In [18]:
df = pd.DataFrame(race_data, columns=column_headers)

In [19]:
df

,Year,CD,Redistricted Date,Total Vote,Vote,Candidate,Vote,Candidate,Vote,Candidate,Other Vote,Plurality,WinningParty,Rep.,Dem.,Highest Other,Other,Rep.,Dem.
0,2014,District 1,01/01/2012,"152,234","103,758","Byrne, Bradley [*]","48,278","LeFlore, Burton R.",198,Write-In(Write-in),N/A,"55,480",R,68.16,31.71,0.13,0,68.25,31.75
1,2012,District 1,01/01/2012,Unopposed,"196,374","Bonner, Josiah Robbins Jr. [*]",N/A,N/A,"4,302",Scattered Write-In(Write-in),N/A,"192,072",R,97.86,0,2.14,0,100,0
2,2014,District 2,01/01/2012,"167,952","113,103","Roby, Martha [*]","54,692","Wright, Erick",157,Write-In(Write-in),N/A,"58,411",R,67.34,32.56,0.09,0,67.41,32.59
3,2012,District 2,01/01/2012,"283,953","180,591","Roby, Martha [*]","103,092","Ford, Therese",270,Scattered Write-In(Write-in),N/A,"77,499",R,63.6,36.31,0.1,0,63.66,36.34
4,2014,District 3,01/01/2012,"156,620","103,558","Rogers, Mike D. [*]","52,816","Smith, Jesse Tremain",246,Write-In(Write-in),N/A,"50,742",R,66.12,33.72,0.16,0,66.22,33.78
5,2012,District 3,01/01/2012,"273,930","175,306","Rogers, Mike D. [*]","98,141","Harris, John Andrew",483,Scattered Write-In(Write-in),N/A,"77,165",R,64,35.83,0.18,0,64.11,35.89
6,2014,District 4,01/01/2012,Unopposed,"132,831","Aderholt, Robert [*]",N/A,N/A,"1,921",Write-In(Write-in),N/A,"130,910",R,98.57,0,1.43,0,100,0
7,2012,District 4,01/01/2012,"269,118","199,071","Aderholt, Robert [*]","69,706","Boman, Daniel H.",341,Scattered Write-In(Write-in),N/A,"129,365",R,73.97,25.9,0.13,0,74.07,25.93
8,2014 [1],District 5,01/01/2012,"154,974","115,338","Brooks, Mo [*]",N/A,N/A,"39,005","Bray, Mark(Unaffiliated)",631,"76,333",R,74.42,0,25.17,0.41,100,0
9,2012,District 5,01/01/2012,"291,293","189,185","Brooks, Mo [*]","101,772","Holley, Charlie L.",336,Scattered Write-In(Write-in),N/A,"87,413",R,64.95,34.94,0.12,0,65.02,34.98


## Munging the Data

We have the raw data from the html into a data frame, now we need to clean the data to ready it for ingestion. 

Delete the empty record at the end. 

In [20]:
df = df[:-1]

In [21]:
df

,Year,CD,Redistricted Date,Total Vote,Vote,Candidate,Vote,Candidate,Vote,Candidate,Other Vote,Plurality,WinningParty,Rep.,Dem.,Highest Other,Other,Rep.,Dem.
0,2014,District 1,01/01/2012,"152,234","103,758","Byrne, Bradley [*]","48,278","LeFlore, Burton R.",198,Write-In(Write-in),N/A,"55,480",R,68.16,31.71,0.13,0,68.25,31.75
1,2012,District 1,01/01/2012,Unopposed,"196,374","Bonner, Josiah Robbins Jr. [*]",N/A,N/A,"4,302",Scattered Write-In(Write-in),N/A,"192,072",R,97.86,0,2.14,0,100,0
2,2014,District 2,01/01/2012,"167,952","113,103","Roby, Martha [*]","54,692","Wright, Erick",157,Write-In(Write-in),N/A,"58,411",R,67.34,32.56,0.09,0,67.41,32.59
3,2012,District 2,01/01/2012,"283,953","180,591","Roby, Martha [*]","103,092","Ford, Therese",270,Scattered Write-In(Write-in),N/A,"77,499",R,63.6,36.31,0.1,0,63.66,36.34
4,2014,District 3,01/01/2012,"156,620","103,558","Rogers, Mike D. [*]","52,816","Smith, Jesse Tremain",246,Write-In(Write-in),N/A,"50,742",R,66.12,33.72,0.16,0,66.22,33.78
5,2012,District 3,01/01/2012,"273,930","175,306","Rogers, Mike D. [*]","98,141","Harris, John Andrew",483,Scattered Write-In(Write-in),N/A,"77,165",R,64,35.83,0.18,0,64.11,35.89
6,2014,District 4,01/01/2012,Unopposed,"132,831","Aderholt, Robert [*]",N/A,N/A,"1,921",Write-In(Write-in),N/A,"130,910",R,98.57,0,1.43,0,100,0
7,2012,District 4,01/01/2012,"269,118","199,071","Aderholt, Robert [*]","69,706","Boman, Daniel H.",341,Scattered Write-In(Write-in),N/A,"129,365",R,73.97,25.9,0.13,0,74.07,25.93
8,2014 [1],District 5,01/01/2012,"154,974","115,338","Brooks, Mo [*]",N/A,N/A,"39,005","Bray, Mark(Unaffiliated)",631,"76,333",R,74.42,0,25.17,0.41,100,0
9,2012,District 5,01/01/2012,"291,293","189,185","Brooks, Mo [*]","101,772","Holley, Charlie L.",336,Scattered Write-In(Write-in),N/A,"87,413",R,64.95,34.94,0.12,0,65.02,34.98


Fix the columns - combine the top level headers with the sub-headers, add a column for state (and fill in accordingly), delete asterisks from candidate name fields, delete the word district. 

In [24]:
#add a state column, fill in with AL
df.add(constant, axis='columns', level=None, fill_value=None)


NameError: name 'constant' is not defined

In [ ]:
#remove asterisks

In [ ]:
#remove "district"